# NPM Supply Chain Network Analysis
## Automated Analysis Pipeline

This notebook performs a comprehensive analysis of the NPM supply chain network. It executes the following steps:
1.  **Data Acquisition**: Fetches the most dependent packages from `ecosyste.ms`.
2.  **Network Construction**: Builds a dependency graph using `networkx`.
3.  **Risk Assessment**: Calculates centrality metrics and a composite **Behavioral Risk Score (BRS)**.
4.  **Robustness Simulation**: Simulates targeted and random attacks to evaluate network resilience.
5.  **Visualization**: Generates plots for network topology, degree distributions, and simulation results.
6.  **Strategic Insights**: Analyzes risk components and ecosystem relationships.
7.  **Validation**: Validates the BRS model against cascade impact simulations.
8.  **Export**: Saves results for external analysis.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import sys
import os

# Add current directory to path to ensure imports work
sys.path.append(os.getcwd())

# Import custom modules
from data_loader import get_combined_seed_packages
from network_builder import build_dependency_graph
from metrics import calculate_risk_scores
from simulation import simulate_attacks, calculate_single_node_impact
from utils import export_results
from visualize import (
    plot_network_structure, 
    plot_degree_distributions, 
    plot_correlations, 
    plot_top_risk_scores, 
    plot_simulation_results,
    plot_risk_vs_cascade,
    plot_metric_heatmap, 
    plot_brs_components, 
    plot_ecosystem_vs_network, 
    plot_risk_distribution_by_type,
    plot_brs_vs_cascade_validation, 
    plot_brs_distribution, 
    plot_top20_brs_component_lines
)

# --- Configuration ---
TOP_N = 1000              # Number of seed packages to fetch (per category)
MAX_DEPTH = 7           # Traversal depth for the crawler
BETWEENNESS_K = 100     # Sampling size for faster betweenness calculation
API_DELAY = 0.1         # Delay between API calls in seconds
OUTPUT_DIR = '../results'  # Output directory for verification run

# --- Visualization Setup ---
sns.set_theme(style="whitegrid", context="notebook", palette="deep")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

## 2. Data Acquisition
We fetch the top `TOP_N` packages sorted by both **dependents** and **downloads** from the `ecosyste.ms` API. These packages serve as the "seed" nodes for our network crawler.

Using a combined strategy ensures we capture both:
1.  **Infrastructure Giants**: Packages that other packages depend on (High Dependents).
2.  **End-User Tools**: Packages that developers use directly (High Downloads).

In [ ]:
# Fetch seed packages (Combined Strategy)
seed_packages = get_combined_seed_packages(limit_dependents=TOP_N, limit_downloads=TOP_N, api_delay=API_DELAY)
print(f"Total unique seed packages: {len(seed_packages)}")

## 3. Network Construction
Using the seed packages, we build a directed dependency graph. The crawler traverses dependencies up to `MAX_DEPTH`.
- **Nodes**: Packages
- **Edges**: Directed dependencies (Package A depends on Package B)


In [ ]:
# Build Graph
G = build_dependency_graph(seed_packages, max_depth=MAX_DEPTH, api_delay=API_DELAY)

In [ ]:
# Print Graph Statistics
print(f"Graph Statistics:")
print(f" - Nodes: {G.number_of_nodes()}")
print(f" - Edges: {G.number_of_edges()}")
print(f" - Density: {nx.density(G):.6f}")
print(f" - Is Strongly Connected: {nx.is_strongly_connected(G)}")
print(f" - Is Weakly Connected: {nx.is_weakly_connected(G)}")


## 4. Metrics & Risk Scoring
We calculate centrality metrics to identify critical nodes:
- **In-Degree**: Number of packages depending on a node within the network (Popularity in the backbone).
- **Out-Degree**: Number of dependencies a node has (Complexity and propagation potential).
- **Betweenness Centrality**: Frequency of a node appearing on shortest paths (Bridge/Bottleneck role).
- **Clustering Coefficient**: Local connectivity (Low clustering = Structural Hole/Bridge = High Risk).
- **Dependents Count**: Number of packages depending on this node in the entire NPM ecosystem.
- **Downloads**: Number of downloads (Usage intensity).
- **Staleness**: Days since last update (Abandonware risk).

**Behavioral Risk Score (BRS) Formula (Balanced Model):**

*   **Structure (40%)**: Betweenness (20%) + Clustering (10%) + Out-Degree (10%)
*   **Lifecycle (30%)**: Staleness (20%) + Maintainer Risk (10%)
*   **Popularity (30%)**: In-Degree (10%) + Dependents (10%) + Downloads (10%)

$$BRS = 0.20 \times Betweenness + 0.10 \times Clustering + 0.10 \times OutDegree + 0.20 \times Staleness + 0.10 \times Maintainer + 0.10 \times InDegree + 0.10 \times Dependents + 0.10 \times Downloads$$

All metrics are normalized (0-1) before calculation. Skewed metrics like Downloads and Dependents use log-normalization.

In [ ]:
# Calculate Metrics
risk_df = calculate_risk_scores(G, betweenness_k=BETWEENNESS_K)
print("\nTop 10 Critical Packages:")
display(risk_df.head(10))

## 5. Robustness Simulation
We simulate supply chain attacks by removing nodes and measuring the network's fragmentation.
- **Targeted Attack**: Removing nodes with the highest BRS.
- **Random Attack**: Removing random nodes.
- **Metric**: Size of the Largest Connected Component (LCC). A faster decay in LCC indicates higher vulnerability.

We also calculate the **Cascade Impact** for the top risky nodes, defined as the reduction in LCC size when that single node is removed.


In [ ]:
# Run Simulations
print("Running robustness simulations...")
sim_results = simulate_attacks(G, risk_df, num_removals=50)

# Calculate Cascade Impact
print("Calculating cascade impact for top nodes...")
impact_df = calculate_single_node_impact(G, risk_df, sample_size=100)
print("Top 5 Packages by Cascade Impact:")
display(impact_df.head(5))

## 6. Core Network Metrics
Here we visualize the fundamental structure of the network and the distribution of node degrees.
- **Network Topology**: A visual representation of the graph (sampled if too large).
- **Degree Distributions**: Histograms showing In-Degree and Out-Degree distributions (typically power-law).
- **Correlations**: A scatter matrix showing relationships between different risk metrics.
- **Top Risk Scores**: A bar chart of the packages with the highest BRS.

In [ ]:
# 1. Network Topology
plot_network_structure(G, output_dir=OUTPUT_DIR + '/plots')

# 2. Degree Distributions
plot_degree_distributions(G, output_dir=OUTPUT_DIR + '/plots')

# 3. Correlations (Metrics)
plot_correlations(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 4. Top Risk Scores
plot_top_risk_scores(risk_df, output_dir=OUTPUT_DIR + '/plots')

## 7. Robustness & Cascade Analysis
This section focuses on the network's resilience to attacks.
- **Risk vs Cascade Impact**: Scatter plot showing if high-risk nodes actually cause high damage when removed.
- **Simulation Results**: Line chart comparing network degradation (LCC size) under Targeted vs Random attacks.

In [ ]:
# 5. Risk vs Cascade Impact Correlation
plot_risk_vs_cascade(impact_df, output_dir=OUTPUT_DIR + '/plots')

# 6. Simulation Results (Robustness)
plot_simulation_results(sim_results, output_dir=OUTPUT_DIR + '/plots')

## 8. Strategic Insights
Advanced visualizations to understand the multidimensional nature of supply chain risk.
- **Multi-metric Heatmap**: Visualizes how top packages score across different metrics (Popularity, Structure, Lifecycle).
- **BRS Components**: Stacked bar chart showing which factors contribute most to the risk score of critical packages.
- **Ecosystem vs Network**: Compares a package's global popularity (NPM dependents) vs its structural centrality in our graph.
- **Risk Distribution by Type**: Shows if "discovered" packages (found via crawling) are as risky as the initial "seed" packages.

In [ ]:
# --- NEW VISUALIZATIONS: Strategic Insights ---

# 7. Multi-metric Heatmap (Shows complexity of risk assessment)
plot_metric_heatmap(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 8. BRS Components Breakdown (Shows contribution of each factor)
plot_brs_components(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 9. Ecosystem vs Network Analysis (Backbone vs broader ecosystem)
plot_ecosystem_vs_network(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 10. Risk Distribution by Package Type (Seed vs discovered)
plot_risk_distribution_by_type(risk_df, output_dir=OUTPUT_DIR + '/plots')

## 9. Validation of BRS Model
These plots prove that our **Balanced Risk Score (BRS)** is a scientifically valid metric.
- **BRS vs Cascade Impact**: The "Proof of Concept" plot. A strong correlation here proves that BRS predicts real-world damage.
- **BRS Distribution**: Checks if the risk scores follow a realistic distribution (Power-Law / Heavy-Tailed), which is expected in scale-free networks.
- **Component Alignment**: Shows how individual metrics align with the actual impact line.

In [ ]:
# --- VALIDATION PLOTS: Proving BRS Formula Effectiveness ---

# 11. BRS vs Cascade Impact Validation (CRITICAL: Shows BRS predicts real network damage)
plot_brs_vs_cascade_validation(impact_df, output_dir=OUTPUT_DIR + '/plots')

# 12. BRS Distribution Analysis (Shows realistic, power-law characteristics)
plot_brs_distribution(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 13. Top 20 BRS Component Lines Chart (Shows how BRS components align with actual cascade impact)
plot_top20_brs_component_lines(risk_df, impact_df, output_dir=OUTPUT_DIR + '/plots')

## 10. Export & Conclusion
Finally, we export all calculated metrics and the graph structure for external use (e.g., Gephi, Excel).
- **CSV Exports**: `risk_scores.csv`, `gephi_nodes.csv`, `gephi_edges.csv`.
- **GML Export**: `network.gml` for graph visualization software.
- **Summary**: Prints final statistics of the analysis run.

In [ ]:
# 14. Export Data
export_results(risk_df, G, impact_df=impact_df, output_dir=OUTPUT_DIR)

# 15. Save Graph Object
nx.write_gml(G, OUTPUT_DIR + "/network.gml")
print(f" - Saved {OUTPUT_DIR}/network.gml")

print("\n" + "="*70)
print("ANALYSIS COMPLETE!")
print("="*70)
print(f"All visualizations and validation plots saved to {OUTPUT_DIR}/plots/")
print(f"Total packages analyzed: {len(risk_df)}")
print(f"Network nodes: {G.number_of_nodes()}, edges: {G.number_of_edges()}")
print("Total plots generated: 13 comprehensive visualizations")
print("="*70)